# 马尔可夫决策过程与动态规划

---

## 学习目标

通过本教程，你将学会：
- 理解马尔可夫决策过程 (MDP) 的数学定义
- 掌握贝尔曼方程（期望与最优）
- 实现策略评估 (Policy Evaluation)
- 实现策略迭代 (Policy Iteration)
- 实现值迭代 (Value Iteration)
- 在网格世界环境中验证算法

## 前置知识

- Python 基础语法
- NumPy 基本操作
- 概率论基础（条件概率、期望）
- 线性代数基础

## 预计时间

45-60 分钟

---

## 第1部分：理论背景

### 什么是强化学习？

强化学习是机器学习的三大范式之一，研究智能体 (Agent) 如何在与环境 (Environment) 的交互中学习最优行为策略。

```
              动作 a_t
     ┌────────────────────┐
     │                    ▼
┌─────────┐          ┌─────────┐
│  Agent  │          │   Env   │
│ (智能体) │          │  (环境)  │
└─────────┘          └─────────┘
     ▲                    │
     │   状态 s_{t+1}     │
     │   奖励 r_{t+1}     │
     └────────────────────┘
```

### 马尔可夫决策过程 (MDP)

MDP 是强化学习的数学框架，由五元组定义：

$$\mathcal{M} = \langle \mathcal{S}, \mathcal{A}, P, R, \gamma \rangle$$

| 符号 | 名称 | 定义 |
|------|------|------|
| $\mathcal{S}$ | 状态空间 | 所有可能状态的集合 |
| $\mathcal{A}$ | 动作空间 | 所有可能动作的集合 |
| $P$ | 转移函数 | $P(s' \mid s, a) = \Pr(S_{t+1}=s' \mid S_t=s, A_t=a)$ |
| $R$ | 奖励函数 | $R(s, a, s')$ 或 $R(s, a)$ |
| $\gamma$ | 折扣因子 | $\gamma \in [0, 1]$ |

### 马尔可夫性质

核心假设：未来状态仅依赖当前状态，与历史无关。

$$P(S_{t+1} \mid S_t, A_t, S_{t-1}, A_{t-1}, \ldots) = P(S_{t+1} \mid S_t, A_t)$$

### 策略与价值函数

策略 $\pi$: 定义智能体在各状态下的行为方式
- 随机策略: $\pi(a \mid s) = \Pr(A_t = a \mid S_t = s)$
- 确定性策略: $a = \pi(s)$

状态价值函数 $V^\pi(s)$:
$$V^\pi(s) = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s\right]$$

动作价值函数 $Q^\pi(s, a)$:
$$Q^\pi(s, a) = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s, A_t = a\right]$$

### 贝尔曼方程

贝尔曼期望方程（给定策略）:
$$V^\pi(s) = \sum_{a} \pi(a \mid s) \left[ R(s, a) + \gamma \sum_{s'} P(s' \mid s, a) V^\pi(s') \right]$$

贝尔曼最优方程:
$$V^*(s) = \max_a \left[ R(s, a) + \gamma \sum_{s'} P(s' \mid s, a) V^*(s') \right]$$

直觉理解: 当前价值 = 即时奖励 + 折扣后的后继状态价值

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from IPython.display import display, HTML

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 可视化配置
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)

print("库导入和配置完成")

### 步骤2: 定义网格世界环境

我们创建一个 4×4 的网格世界：
- **S**: 起点 (0,0)
- **G**: 目标 (3,3)
- **X**: 障碍物
- 每步移动获得 -1 奖励（鼓励快速到达目标）
- 到达目标获得 +100 奖励

In [ ]:
# ============================================================
# 网格世界配置
# ============================================================

@dataclass
class GridConfig:
    """网格世界配置"""
    size: int = 4
    start: Tuple[int, int] = (0, 0)
    goal: Tuple[int, int] = (3, 3)
    obstacles: List[Tuple[int, int]] = None
    step_reward: float = -1.0
    goal_reward: float = 100.0
    slip_prob: float = 0.0  # 滑动概率（随机性）
    
    def __post_init__(self):
        if self.obstacles is None:
            self.obstacles = [(1, 1)]  # 默认障碍物

# 创建默认配置
config = GridConfig()

print(f"网格大小: {config.size}×{config.size}")
print(f"起点: {config.start}")
print(f"目标: {config.goal}")
print(f"障碍物: {config.obstacles}")
print(f"每步奖励: {config.step_reward}")
print(f"目标奖励: {config.goal_reward}")

In [ ]:
# ============================================================
# 网格世界环境类
# ============================================================

class GridWorld:
    """
    网格世界 MDP 环境
    
    状态: (行, 列) 元组
    动作: '上', '下', '左', '右'
    """
    
    # 动作到坐标变化的映射
    ACTION_DELTAS = {
        '上': (-1, 0),
        '下': (1, 0),
        '左': (0, -1),
        '右': (0, 1)
    }
    
    def __init__(self, config: GridConfig):
        self.config = config
        self.size = config.size
        self.goal = config.goal
        self.obstacles = set(config.obstacles)
        
        # 构建状态空间（排除障碍物）
        self.states = [
            (i, j) for i in range(self.size) 
            for j in range(self.size)
            if (i, j) not in self.obstacles
        ]
        
        # 动作空间
        self.actions = list(self.ACTION_DELTAS.keys())
        
    def is_terminal(self, state: Tuple[int, int]) -> bool:
        """判断是否为终止状态"""
        return state == self.goal
    
    def get_next_state(self, state: Tuple[int, int], action: str) -> Tuple[int, int]:
        """计算下一状态（处理边界和障碍物碰撞）"""
        if self.is_terminal(state):
            return state
            
        di, dj = self.ACTION_DELTAS[action]
        ni = max(0, min(self.size - 1, state[0] + di))
        nj = max(0, min(self.size - 1, state[1] + dj))
        next_state = (ni, nj)
        
        # 碰到障碍物则停留原地
        if next_state in self.obstacles:
            return state
        return next_state
    
    def get_reward(self, state: Tuple[int, int], next_state: Tuple[int, int]) -> float:
        """获取奖励"""
        if next_state == self.goal:
            return self.config.goal_reward
        return self.config.step_reward
    
    def get_transitions(self, state: Tuple[int, int], action: str) -> List[Tuple[Tuple[int, int], float, float]]:
        """
        获取状态转移分布
        
        Returns:
            [(next_state, probability, reward), ...]
        """
        if self.is_terminal(state):
            return [(state, 1.0, 0.0)]
        
        next_state = self.get_next_state(state, action)
        reward = self.get_reward(state, next_state)
        return [(next_state, 1.0, reward)]  # 确定性环境

# 创建环境
env = GridWorld(config)

print(f"\n环境创建成功")
print(f"状态空间大小: {len(env.states)}")
print(f"动作空间: {env.actions}")
print(f"状态列表: {env.states}")

### 步骤3: 可视化网格世界

In [ ]:
# ============================================================
# 可视化函数
# ============================================================

def visualize_grid(env: GridWorld, title: str = "Grid World"):
    """可视化网格世界"""
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # 绘制网格
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    # 标记特殊位置
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) == env.config.start:
                ax.text(j + 0.5, env.size - i - 0.5, 'S', 
                       ha='center', va='center', fontsize=20, fontweight='bold', color='blue')
            elif (i, j) == env.goal:
                ax.text(j + 0.5, env.size - i - 0.5, 'G', 
                       ha='center', va='center', fontsize=20, fontweight='bold', color='green')
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, 
                                          facecolor='lightgreen', alpha=0.5))
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, 
                                          facecolor='gray', alpha=0.8))
                ax.text(j + 0.5, env.size - i - 0.5, 'X', 
                       ha='center', va='center', fontsize=20, fontweight='bold', color='white')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()

visualize_grid(env, "网格世界环境")

---

## 第3部分：动态规划算法

### 3.1 策略评估 (Policy Evaluation)

给定策略 $\pi$，迭代计算其价值函数：

$$V_{k+1}(s) = \sum_{a} \pi(a \mid s) \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V_k(s')]$$

算法背景：策略评估由Richard Bellman提出，是动态规划的基础算子，用于评估给定策略的价值。通过迭代求解贝尔曼期望方程，将当前价值估计代入方程右侧产生新估计，直到收敛。这是一个不动点迭代过程，保证线性收敛到策略的真实价值函数。

In [ ]:
# ============================================================
# 策略评估算法
# ============================================================

def policy_evaluation(
    env: GridWorld,
    policy: Dict[Tuple[int, int], Dict[str, float]],
    gamma: float = 0.99,
    theta: float = 1e-6,
    max_iterations: int = 10000
) -> Tuple[Dict[Tuple[int, int], float], int]:
    """
    策略评估：计算给定策略的状态价值函数
    
    Args:
        env: 环境
        policy: 策略 π(a|s)
        gamma: 折扣因子
        theta: 收敛阈值
        max_iterations: 最大迭代次数
        
    Returns:
        (V, iterations): 价值函数和迭代次数
    """
    # 初始化价值函数
    V = {s: 0.0 for s in env.states}
    
    for iteration in range(1, max_iterations + 1):
        delta = 0.0
        
        for state in env.states:
            if env.is_terminal(state):
                continue
                
            old_value = V[state]
            new_value = 0.0
            
            # 贝尔曼期望方程
            for action in env.actions:
                action_prob = policy.get(state, {}).get(action, 0.0)
                if action_prob > 0:
                    for next_state, trans_prob, reward in env.get_transitions(state, action):
                        new_value += action_prob * trans_prob * (reward + gamma * V.get(next_state, 0.0))
            
            V[state] = new_value
            delta = max(delta, abs(old_value - new_value))
        
        if delta < theta:
            return V, iteration
    
    return V, max_iterations

print("策略评估函数定义完成")

In [ ]:
# ============================================================
# 测试策略评估：均匀随机策略
# ============================================================

# 创建均匀随机策略
random_policy = {
    s: {a: 0.25 for a in env.actions}
    for s in env.states
}

print("随机策略示例（每个动作概率相等）:")
print(f"  状态 (0,0): {random_policy[(0, 0)]}")

# 执行策略评估
V_random, iterations = policy_evaluation(env, random_policy, gamma=0.99)

print(f"\n策略评估完成，迭代次数: {iterations}")
print("\n状态价值函数 V^π(s):")
for i in range(env.size):
    row = []
    for j in range(env.size):
        if (i, j) in V_random:
            row.append(f"{V_random[(i, j)]:7.2f}")
        elif (i, j) in env.obstacles:
            row.append("   X   ")
        else:
            row.append("   -   ")
    print(" | ".join(row))

### 3.2 策略改进 (Policy Improvement)

基于当前价值函数贪婪改进策略：

$$\pi'(s) = \arg\max_a \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V(s')]$$

算法背景：策略改进定理由Bellman提出，保证贪婪策略至少不比原策略差。该定理是策略迭代收敛性的理论基础。策略改进将价值信息转化为行动决策，选择使Q值最大的动作来改进策略。

In [ ]:
# ============================================================
# 策略改进算法
# ============================================================

def policy_improvement(
    env: GridWorld,
    V: Dict[Tuple[int, int], float],
    gamma: float = 0.99
) -> Dict[Tuple[int, int], Dict[str, float]]:
    """
    策略改进：基于价值函数贪婪构造新策略
    
    Args:
        env: 环境
        V: 状态价值函数
        gamma: 折扣因子
        
    Returns:
        改进后的确定性策略
    """
    policy = {}
    
    for state in env.states:
        if env.is_terminal(state):
            # 终止状态：均匀随机策略
            policy[state] = {a: 0.25 for a in env.actions}
            continue
        
        # 计算各动作的 Q 值
        q_values = {}
        for action in env.actions:
            q_val = 0.0
            for next_state, trans_prob, reward in env.get_transitions(state, action):
                q_val += trans_prob * (reward + gamma * V.get(next_state, 0.0))
            q_values[action] = q_val
        
        # 选择最优动作
        best_value = max(q_values.values())
        best_actions = [a for a, v in q_values.items() if abs(v - best_value) < 1e-9]
        
        # 确定性策略
        policy[state] = {
            a: 1.0 if a == best_actions[0] else 0.0
            for a in env.actions
        }
    
    return policy

print("策略改进函数定义完成")

### 3.3 策略迭代 (Policy Iteration)

交替执行策略评估和策略改进，直到策略稳定：

```
π₀ → V^π₀ → π₁ → V^π₁ → π₂ → ... → π* → V*
```

算法背景：策略迭代由Ronald Howard于1960年提出，是求解MDP的经典方法。该算法将复杂的最优化问题分解为两个子问题：评估（给定策略求价值）和改进（给定价值求策略）。策略改进定理保证价值单调递增，有限MDP中必收敛到全局最优。通常3-10次外层迭代即可收敛。

In [ ]:
# ============================================================
# 策略迭代算法
# ============================================================

def policy_iteration(
    env: GridWorld,
    gamma: float = 0.99,
    max_iterations: int = 100,
    verbose: bool = True
) -> Tuple[Dict, Dict, int]:
    """
    策略迭代算法
    
    Args:
        env: 环境
        gamma: 折扣因子
        max_iterations: 最大迭代次数
        verbose: 是否打印进度
        
    Returns:
        (policy, V, iterations)
    """
    # 初始化均匀随机策略
    policy = {s: {a: 0.25 for a in env.actions} for s in env.states}
    V = {}
    
    for iteration in range(1, max_iterations + 1):
        # 策略评估
        V, eval_iters = policy_evaluation(env, policy, gamma)
        
        if verbose:
            print(f"迭代 {iteration}: 策略评估用了 {eval_iters} 次内层迭代")
        
        # 策略改进
        new_policy = policy_improvement(env, V, gamma)
        
        # 检查收敛
        if policy == new_policy:
            if verbose:
                print(f"\n✓ 策略迭代收敛，总迭代: {iteration}")
            return new_policy, V, iteration
        
        policy = new_policy
    
    return policy, V, max_iterations

print("\n" + "="*50)
print("运行策略迭代")
print("="*50)

policy_pi, V_pi, iters_pi = policy_iteration(env, gamma=0.99)

### 3.4 值迭代 (Value Iteration)

直接迭代贝尔曼最优方程：

$$V_{k+1}(s) = \max_a \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V_k(s')]$$

算法背景：值迭代由Bellman在1950年代提出，跳过显式的策略维护，直接迭代价值函数。可视为策略迭代的变体，将策略评估截断为单步更新。每次更新隐式地进行策略改进（选择max），因此不需要显式维护策略。基于Banach不动点定理，贝尔曼最优算子是γ-压缩映射，保证收敛到唯一不动点。

In [ ]:
# ============================================================
# 值迭代算法
# ============================================================

def value_iteration(
    env: GridWorld,
    gamma: float = 0.99,
    theta: float = 1e-6,
    max_iterations: int = 10000,
    verbose: bool = True
) -> Tuple[Dict, Dict, int]:
    """
    值迭代算法
    
    Args:
        env: 环境
        gamma: 折扣因子
        theta: 收敛阈值
        max_iterations: 最大迭代次数
        verbose: 是否打印进度
        
    Returns:
        (policy, V, iterations)
    """
    V = {s: 0.0 for s in env.states}
    
    for iteration in range(1, max_iterations + 1):
        delta = 0.0
        
        for state in env.states:
            if env.is_terminal(state):
                continue
            
            old_value = V[state]
            
            # 贝尔曼最优方程
            q_values = []
            for action in env.actions:
                q_val = 0.0
                for next_state, trans_prob, reward in env.get_transitions(state, action):
                    q_val += trans_prob * (reward + gamma * V.get(next_state, 0.0))
                q_values.append(q_val)
            
            V[state] = max(q_values)
            delta = max(delta, abs(old_value - V[state]))
        
        if delta < theta:
            if verbose:
                print(f"✓ 值迭代收敛，迭代次数: {iteration}")
            policy = policy_improvement(env, V, gamma)
            return policy, V, iteration
    
    policy = policy_improvement(env, V, gamma)
    return policy, V, max_iterations

print("\n" + "="*50)
print("运行值迭代")
print("="*50)

policy_vi, V_vi, iters_vi = value_iteration(env, gamma=0.99)

---

## 📊 第4部分：结果可视化

### 可视化策略和价值函数

In [ ]:
# ============================================================
# 策略和价值函数可视化
# ============================================================

def visualize_policy_and_values(
    env: GridWorld,
    policy: Dict,
    V: Dict,
    title: str = "Policy & Value Function"
):
    """可视化策略和价值函数"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 动作箭头符号
    arrow_map = {'上': '↑', '下': '↓', '左': '←', '右': '→'}
    
    # ===== 左图：策略 =====
    ax = axes[0]
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    for i in range(env.size):
        for j in range(env.size):
            y = env.size - i - 0.5
            x = j + 0.5
            
            if (i, j) == env.goal:
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, facecolor='lightgreen', alpha=0.5))
                ax.text(x, y, 'G', ha='center', va='center', fontsize=16, fontweight='bold', color='green')
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, facecolor='gray', alpha=0.8))
                ax.text(x, y, 'X', ha='center', va='center', fontsize=16, fontweight='bold', color='white')
            elif (i, j) in policy:
                best_action = max(policy[(i, j)], key=policy[(i, j)].get)
                ax.text(x, y, arrow_map[best_action], ha='center', va='center', fontsize=20, color='blue')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title('最优策略 π*', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # ===== 右图：价值函数 =====
    ax = axes[1]
    
    # 创建价值矩阵用于颜色映射
    value_matrix = np.zeros((env.size, env.size))
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) in V:
                value_matrix[i, j] = V[(i, j)]
            else:
                value_matrix[i, j] = np.nan
    
    im = ax.imshow(value_matrix, cmap='RdYlGn', aspect='equal')
    
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) == env.goal:
                ax.text(j, i, 'G', ha='center', va='center', fontsize=14, fontweight='bold')
            elif (i, j) in env.obstacles:
                ax.text(j, i, 'X', ha='center', va='center', fontsize=14, fontweight='bold', color='white')
            elif (i, j) in V:
                ax.text(j, i, f'{V[(i, j)]:.1f}', ha='center', va='center', fontsize=10)
    
    ax.set_title('状态价值函数 V*', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.colorbar(im, ax=ax, shrink=0.8)
    
    fig.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

# 可视化策略迭代结果
visualize_policy_and_values(env, policy_pi, V_pi, "策略迭代结果")

# 可视化值迭代结果
visualize_policy_and_values(env, policy_vi, V_vi, "值迭代结果")

### 算法对比

In [ ]:
# ============================================================
# 算法对比
# ============================================================

print("="*60)
print("策略迭代 vs 值迭代")
print("="*60)
print(f"{'算法':<20} {'外层迭代':>15} {'特点':>20}")
print("-"*60)
print(f"{'策略迭代':<20} {iters_pi:>15} {'精确评估，迭代少':>20}")
print(f"{'值迭代':<20} {iters_vi:>15} {'截断评估，每次代价低':>20}")
print("-"*60)

# 验证两种算法产生相同的价值函数
value_diff = sum(abs(V_pi[s] - V_vi[s]) for s in env.states)
print(f"\n价值函数总差异: {value_diff:.10f}")

if value_diff < 1e-5:
    print("✓ 两种算法收敛到相同的最优价值函数！")

---

## 🎮 第5部分：交互式实验

### 实验1：改变折扣因子 γ

In [ ]:
# ============================================================
# 实验：不同折扣因子的影响
# ============================================================

gammas = [0.5, 0.9, 0.99]

print("不同折扣因子 γ 对价值函数的影响:")
print("="*60)

for gamma in gammas:
    _, V_gamma, iters = value_iteration(env, gamma=gamma, verbose=False)
    
    print(f"\nγ = {gamma}, 收敛迭代: {iters}")
    print(f"起点 V(0,0) = {V_gamma[(0, 0)]:.2f}")
    print(f"目标邻近 V(3,2) = {V_gamma[(3, 2)]:.2f}")

print("\n" + "="*60)
print("观察：γ 越大，远离目标的状态价值越高（更重视长期回报）")

### 实验2：执行最优策略

In [ ]:
# ============================================================
# 执行最优策略
# ============================================================

def execute_policy(
    env: GridWorld,
    policy: Dict,
    max_steps: int = 50
) -> Tuple[float, List[Tuple[int, int]]]:
    """
    执行策略并返回轨迹
    
    Returns:
        (total_reward, trajectory)
    """
    state = env.config.start
    trajectory = [state]
    total_reward = 0.0
    
    for step in range(max_steps):
        if env.is_terminal(state):
            break
        
        # 选择最优动作
        best_action = max(policy[state], key=policy[state].get)
        
        # 执行动作
        transitions = env.get_transitions(state, best_action)
        next_state, _, reward = transitions[0]
        
        total_reward += reward
        state = next_state
        trajectory.append(state)
    
    return total_reward, trajectory

# 执行最优策略
total_reward, trajectory = execute_policy(env, policy_vi)

print("执行最优策略:")
print(f"总奖励: {total_reward:.1f}")
print(f"步数: {len(trajectory) - 1}")
print(f"轨迹: {' → '.join([str(s) for s in trajectory])}")

In [ ]:
# ============================================================
# 可视化轨迹
# ============================================================

def visualize_trajectory(env: GridWorld, trajectory: List[Tuple[int, int]]):
    """可视化执行轨迹"""
    fig, ax = plt.subplots(figsize=(7, 7))
    
    # 绘制网格
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    # 标记特殊位置
    for i in range(env.size):
        for j in range(env.size):
            y = env.size - i - 0.5
            x = j + 0.5
            
            if (i, j) == env.goal:
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, facecolor='lightgreen', alpha=0.5))
                ax.text(x, y, 'G', ha='center', va='center', fontsize=16, fontweight='bold', color='green')
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size - i - 1), 1, 1, facecolor='gray', alpha=0.8))
    
    # 绘制轨迹
    for idx, (i, j) in enumerate(trajectory):
        y = env.size - i - 0.5
        x = j + 0.5
        
        if idx == 0:
            ax.plot(x, y, 'bo', markersize=15, label='起点')
        elif idx == len(trajectory) - 1:
            ax.plot(x, y, 'g*', markersize=20, label='终点')
        else:
            ax.plot(x, y, 'r.', markersize=10)
    
    # 连接轨迹点
    xs = [t[1] + 0.5 for t in trajectory]
    ys = [env.size - t[0] - 0.5 for t in trajectory]
    ax.plot(xs, ys, 'r-', linewidth=2, alpha=0.7, label='轨迹')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title('最优策略执行轨迹', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()

visualize_trajectory(env, trajectory)

---

## 总结

### 关键要点

1. MDP 五元组: $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$ 完整描述强化学习问题

2. 马尔可夫性质: 未来仅依赖当前状态，与历史无关

3. 贝尔曼方程: 价值函数的递归分解
   - 期望方程: 给定策略的价值
   - 最优方程: 最优价值

4. 策略迭代: 评估 → 改进 → 评估 → ... → 收敛

5. 值迭代: 直接迭代贝尔曼最优方程

### 算法对比

| 算法 | 每次迭代 | 迭代次数 | 适用场景 |
|------|----------|----------|----------|
| 策略迭代 | 完整策略评估 | 少（3-10次） | 小状态空间 |
| 值迭代 | 单步贝尔曼更新 | 多（数百次） | 大状态空间 |

策略迭代：少量外层迭代 × 昂贵的内层评估  
值迭代：大量外层迭代 × 便宜的单步更新

### 动态规划的局限

- 需要完整的环境模型 (P, R)
- 状态空间爆炸问题
- 无法处理连续状态/动作空间

### 下一步学习

- Q-Learning: 无模型方法，通过采样学习
- SARSA: 在策略 TD 学习
- DQN: 深度强化学习，函数逼近
- 策略梯度: 直接优化策略（REINFORCE, PPO, SAC）

### 练习题

1. 增加更多障碍物，观察策略变化
2. 添加随机性 (slip_prob > 0)，比较确定性和随机环境
3. 实现 Q-Learning 算法并与 DP 方法比较

## 参考资料

- Sutton & Barto, "Reinforcement Learning: An Introduction" (2nd ed.), Chapter 4
- Bellman, R. "Dynamic Programming", Princeton University Press, 1957
- Howard, R. "Dynamic Programming and Markov Processes", MIT Press, 1960
- David Silver, UCL RL Course, Lecture 2-3

---

## 🧪 单元测试 | Unit Tests

运行以下测试验证实现的正确性：

In [ ]:
# ============================================================
# 单元测试
# ============================================================

def run_tests():
    """运行所有单元测试"""
    print("开始单元测试...\n")
    passed = 0
    failed = 0
    
    # 测试1: 环境状态空间
    try:
        assert len(env.states) == 15, f"状态数错误: {len(env.states)}"
        assert (1, 1) not in env.states, "障碍物不应在状态空间中"
        print("测试1通过: 状态空间正确")
        passed += 1
    except AssertionError as e:
        print(f"测试1失败: {e}")
        failed += 1
    
    # 测试2: 终止状态
    try:
        assert env.is_terminal((3, 3)) == True, "目标应为终止状态"
        assert env.is_terminal((0, 0)) == False, "起点不应为终止状态"
        print("测试2通过: 终止状态判断正确")
        passed += 1
    except AssertionError as e:
        print(f"测试2失败: {e}")
        failed += 1
    
    # 测试3: 状态转移
    try:
        next_s = env.get_next_state((0, 0), '下')
        assert next_s == (1, 0), f"状态转移错误: {next_s}"
        # 边界测试
        next_s = env.get_next_state((0, 0), '上')
        assert next_s == (0, 0), "边界应停留原地"
        print("测试3通过: 状态转移正确")
        passed += 1
    except AssertionError as e:
        print(f"测试3失败: {e}")
        failed += 1
    
    # 测试4: 策略迭代收敛
    try:
        assert iters_pi <= 10, f"策略迭代应该快速收敛: {iters_pi}"
        print(f"测试4通过: 策略迭代在 {iters_pi} 次收敛")
        passed += 1
    except AssertionError as e:
        print(f"测试4失败: {e}")
        failed += 1
    
    # 测试5: 值迭代收敛
    try:
        assert iters_vi <= 100, f"值迭代应该合理收敛: {iters_vi}"
        print(f"测试5通过: 值迭代在 {iters_vi} 次收敛")
        passed += 1
    except AssertionError as e:
        print(f"测试5失败: {e}")
        failed += 1
    
    # 测试6: 两种算法结果一致
    try:
        diff = sum(abs(V_pi[s] - V_vi[s]) for s in env.states)
        assert diff < 1e-5, f"价值函数差异过大: {diff}"
        print("测试6通过: 策略迭代与值迭代结果一致")
        passed += 1
    except AssertionError as e:
        print(f"测试6失败: {e}")
        failed += 1
    
    # 测试7: 最优策略能到达目标
    try:
        reward, traj = execute_policy(env, policy_vi)
        assert traj[-1] == env.goal, "最优策略应该到达目标"
        assert len(traj) <= 10, f"路径应该较短: {len(traj)}"
        print(f"测试7通过: 最优策略在 {len(traj)-1} 步到达目标")
        passed += 1
    except AssertionError as e:
        print(f"测试7失败: {e}")
        failed += 1
    
    # 测试8: 价值函数单调性
    try:
        # 离目标越近，价值越高
        assert V_vi[(3, 2)] > V_vi[(0, 0)], "靠近目标的价值应该更高"
        assert V_vi[(2, 3)] > V_vi[(0, 0)], "靠近目标的价值应该更高"
        print("测试8通过: 价值函数单调性正确")
        passed += 1
    except AssertionError as e:
        print(f"测试8失败: {e}")
        failed += 1
    
    # 总结
    print(f"\n{'='*50}")
    print(f"测试完成: {passed} 通过, {failed} 失败")
    if failed == 0:
        print("所有测试通过！实现正确。")
    else:
        print("存在失败的测试，请检查代码。")
    print(f"{'='*50}")
    
    return failed == 0

# 运行测试
run_tests()